In [1]:
from functools import partial

from transformers import CLIPVisionModel 
import torch
from torch import nn
from torchvision import transforms
from PIL import Image

import torch
import torch.nn as nn
from transformers import CLIPVisionModel
from torchvision import transforms

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')



In [ ]:
train_pixel_img_feature = torch.load('/root/autodl-tmp/Workspace/EEG_caption/ViT-L-14_features_GIT_train.pt')['img_features']# 
test_pixel_img_feature = torch.load('/root/autodl-tmp/Workspace/EEG_caption/ViT-L-14_features_GIT_test.pt')['img_features']# 
train_img_feature = torch.load('/root/autodl-tmp/Workspace/EEG_caption/ViT-H-14_features_train.pt')['img_features'].unsqueeze(1)# 
test_img_feature = torch.load('/root/autodl-tmp/Workspace/EEG_caption/ViT-H-14_features_test.pt')['img_features'].unsqueeze(1)# 


In [ ]:
test_img_feature.shape

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from einops.layers.torch import Rearrange, Reduce

# Define the neural network
class PixelProjector(nn.Sequential):
    def __init__(self, proj_dim=1024):
        super().__init__(
            Rearrange('B C L->B L C'),    
            nn.Linear(1, 257),
            nn.LayerNorm(257),
            Rearrange('B L C->B C L'),
            nn.Linear(1024, 1024),
            nn.LayerNorm(proj_dim),
            )
        
        

# Instantiate the model, loss function, and optimizer

model = PixelProjector(proj_dim=1024).to(torch.bfloat16).to(device)
criterion = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001)

# Prepare data loaders
train_dataset = TensorDataset(train_img_feature, train_pixel_img_feature)
test_dataset = TensorDataset(test_img_feature, test_pixel_img_feature)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Training loop
num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(torch.bfloat16).to(device), targets.to(torch.bfloat16).to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

# Testing loop
model.eval()
test_loss = 0.0
with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(torch.bfloat16).to(device), targets.to(torch.bfloat16).to(device)
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item()

print(f"Test Loss: {test_loss/len(test_loader)}")

# Save the trained model
torch.save(model.state_dict(), '/root/autodl-tmp/Workspace/EEG_caption/model_weights/PixelProjector_best.bin')
print("Model saved as PixelProjector.bin")


In [8]:
# Save the trained model
torch.save(model.state_dict(), '/root/autodl-tmp/Workspace/EEG_caption/model_weights/PixelProjector_best.bin')
print("Model saved as PixelProjector.bin")

Model saved as PixelProjector.bin
